# cjio API tutorial

In this tutorial we explore what is possible with `cjio`'s API. I refer to `cjio`'s command line interface as *CLI*.

The CLI is what you use when you invoke `cjio` from the command line, such as:
```
$ cjio some_city_model.json validate
```

The API is what you use from `cjio` when working with a city model in a Python script.

In [1]:
import os
from copy import deepcopy
from cjio import cityjson

Set up the paths for the tutorial.

In [2]:
package_dir = os.path.dirname('.')
schema_dir = os.path.join(package_dir, 'cjio', 'schemas', '1.0.0')
data_dir = os.path.join(package_dir, 'example_data')

## Load the city model

We are working with a subset of the city model of Rotterdam. This file is included with `cjio` as an example data set. The `load()` method loads a CityJSON file into a CityJSON object.

In [3]:
p_subset = os.path.join(data_dir, 'rotterdam', 'rotterdam_subset.json')
# p_subset = os.path.join(data_dir, 'zurich', 'zurich_subset_lod2.json')
cm_r = cityjson.load(p_subset)
print(type(cm_r))

<class 'cjio.cityjson.CityJSON'>


## Using the CLI commands in the API
You can use any of the CLI commands on a CityJSON object. However, not all CLI commands are mapped 1-to-1 to CityJSON methods, plus you need to provide the parameters yourself, as for example in case of the `validate()` method. The reason for this is that the CLI commands are wrapper functions that take care of passing parameters and parsing outputs. And we haven't harmonized the CLI and the API yet. 

In [4]:
cm_r.validate(folder_schemas=schema_dir)

-- Validating the syntax of the file
	(using the schemas cjio/schemas/1.0.0/cityjson.schema.json)
-- Validating the internal consistency of the file (see docs for list)
	--Vertex indices coherent
	--Specific for CityGroups
	--Semantic arrays coherent with geometry
	--Root properties
	--Empty geometries
	--Duplicate vertices
	--Orphan vertices
	--CityGML attributes


(True,
 False,
 [],
 ["WARNING: attributes 'TerrainHeight' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'bron_tex' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'voll_tex' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'bron_geo' not in CityGML schema",
  '\t(16 CityObjects have this warning)',
  "WARNING: attributes 'status' not in CityGML schema",
  '\t(16 CityObjects have this warning)'])

## Explore the city model

We can print the basic information about the city model. Note that `print()` returns the same information as the `info` command in the CLI.

In [5]:
print(cm_r)

{
  "cityjson_version": "1.0",
  "epsg": 7415,
  "cityobjects_total": 16,
  "cityobjects_present": [
    "Building"
  ],
  "vertices_total": 383,
  "transform/compressed": true,
  "geom_primitives_present": [
    "MultiSurface"
  ],
  "materials": false,
  "textures": true
}


## Working with the objects in the model
### Getting objects from the model

In [6]:
buildings = cm_r.get_cityobjects(type='building')
buildings_parts = cm_r.get_cityobjects(type=['building', 'buildingpart'])

r_ids = ['{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE}',
         '{6271F75F-E8D8-4EE4-AC46-9DB02771A031}']
buildings_ids = cm_r.get_cityobjects(id=r_ids)

### Assigning attributes to Semantic Surfaces
In order to change or assign an attribute to the SemanticSurfaces of a CityObject we need to,

1. extract the surfaces,
2. make the changes on the surface,
3. overwrite the CityObjects with the changes.

In [7]:
cm = deepcopy(cm_r)
new_cos = {}
for co_id, co in cm.cityobjects.items():
    new_geoms = []
    for geom in co.geometry:
        # Only LoD >= 2 models have semantic surfaces
        if geom.lod >= 2.0:
            # Extract the surfaces
            roofsurfaces = geom.get_surfaces('roofsurface')
            for i, rsrf in roofsurfaces.items():
                # Change the attributes
                if 'attributes' in rsrf.keys():
                    rsrf['attributes']['cladding'] = 'tiles'
                else:
                    rsrf['attributes'] = {}
                    rsrf['attributes']['cladding'] = 'tiles'
                geom.surfaces[i] = rsrf
            new_geoms.append(geom)
        else:
            # Use the unchanged geometry
            new_geoms.append(geom)
    co.geometry = new_geoms
    new_cos[co_id] = co
# Overwrite the CityObjects
cm.cityobjects = new_cos

## Save or Export
At the end, the `save()` method saves the edited city model into a CityJSON file.

In [8]:
p_out = os.path.join(data_dir, 'test_output.json')
cityjson.save(cm, p_out)

It is also possible to export the city model into a pandas DataFrame. Note that only the CityObject attributes are exported into the dataframe, with CityObject IDs as the index of the dataframe. Thus if you want to export the attributes of SemanticSurfaces for example, then you need to add them as CityObject attributes.

In [10]:
df = cm.to_dataframe()
df.head()

,TerrainHeight,bron_geo,bron_tex,status,voll_tex
{C9D4A5CF-094A-47DA-97E4-4A3BFD75D3AE},3.03,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,1,complete
{71B60053-BC28-404D-BAB9-8A642AAC0CF4},2.68,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,1,complete
{6271F75F-E8D8-4EE4-AC46-9DB02771A031},3.12,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,1,complete
{DE77E78F-B110-43D2-A55C-8B61911192DE},2.88,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,1,complete
{19935DFC-F7B3-4D6E-92DD-C48EE1D1519A},2.82,Lidar 15-30 punten - nov. 2008,UltraCAM-X 10cm juni 2008,1,complete
